In [1]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import numpy as np

from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

### Sentence Transformer

Sentence Transformers (a.k.a. SBERT) is the go-to Python module for accessing, using, and training state-of-the-art text and image embedding models. Characteristics of Sentence Transformer (a.k.a bi-encoder) models:

1) Calculates a fixed-size vector representation (embedding) given texts or images.
2) Embedding calculation is often efficient, embedding similarity calculation is very fast.
3) Applicable for a wide range of tasks, such as semantic textual similarity, semantic search, clustering, classification, paraphrase mining, and more.
4) Often used as a first step in a two-step retrieval process, where a Cross-Encoder (a.k.a. reranker) model is used to re-rank the top-k results from the bi-encoder.

https://sbert.net/index.html



In [5]:
# creating embedding model from sentence transformer for getting the embeddings of the text

txt_embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [8]:
print(f"Embedding model size: {txt_embedder.get_sentence_embedding_dimension()}")

Embedding model size: 384


### Qdrant setup

Qdrant (read: quadrant) is a vector similarity search engine and vector database. It provides a production-ready service with a convenient API to store, search, and manage points—vectors with an additional payload Qdrant is tailored to extended filtering support. It makes it useful for all sorts of neural-network or semantic-based matching, faceted search, and other applications.

In [6]:
# creating the vector database client using qdrant

qdrant = QdrantClient(":memory:")

In [11]:
# creating Qdrant Collection to store the data

qdrant.recreate_collection(
    collection_name="clinical_notes",
    vectors_config=models.VectorParams(
        size=txt_embedder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

True

### Data Loading

In [12]:
notes_data = pd.read_csv("../data/train.csv")

In [13]:
notes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10822 entries, 0 to 10821
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10822 non-null  int64 
 1   Note    10822 non-null  object
 2   json    10822 non-null  object
dtypes: int64(1), object(2)
memory usage: 253.8+ KB


In [14]:
notes_data.isnull().sum()

ID      0
Note    0
json    0
dtype: int64

In [15]:
notes_data.duplicated().sum()

np.int64(0)

In [16]:
rag_notes = notes_data.sample(1000).to_dict("records")

In [17]:
print(f"Length of the data : {len(rag_notes)}")

Length of the data : 1000


### Vectorize

In [22]:
qdrant.upload_points(
    collection_name="clinical_notes",
    points=[
        models.PointStruct(
            id=d["ID"],
            vector=txt_embedder.encode(d["Note"]).tolist(),
            payload=d
        ) for d in rag_notes
    ]
)

In [24]:
search_prompt = "Suggest me a medication for fever"

In [25]:
# searching for some clinical suggestions

hits = qdrant.search(
    collection_name="clinical_notes",
    query_vector=txt_embedder.encode(search_prompt).tolist(),
    # query_filter=,
    limit=3
)

In [30]:
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'ID': 823, 'Note': '**Clinical Notes:**\n\n**Patient Information:**\n- Age: 88 years\n- Gender: Male\n\n**Visit Motivation:**\n- Patient presents with concerns related to influenza (Flu).\n\n**Symptoms:**\n- Fever: 39.8°C (higher than normal range, indicating a significant febrile response)\n- Cough\n- Fatigue\n- Headache\n- Vomiting\n- Diarrhea\n- Runny nose\n- Rash\n- Increased thirst\n- Heartburn\n- Restlessness\n\n**Vital Signs:**\n- Temperature: 39.8°C (febrile)\n- Cholesterol Level: 162.2 mg/dL (within normal range)\n- Glucose Level: 105.7 mg/dL (slightly elevated, but not critically high)\n\n**Assessment:**\nThe patient is an 88-year-old male presenting with a variety of symptoms consistent with a viral infection, particularly influenza. The high fever suggests a significant immune response, potentially indicating a severe case of influenza or another viral infection. The presence of gastrointestinal symptoms (vomiting and diarrhea) alongside respiratory symptoms (cough, runny 